#Install Dependencies

In [1]:
!git clone https://github.com/krishnap25/casimir
import os
os.chdir("casimir")
os.mkdir("data")

Cloning into 'casimir'...
remote: Enumerating objects: 68, done.
remote: Total 68 (delta 0), reused 0 (delta 0), pack-reused 68
Unpacking objects: 100% (68/68), done.


In [2]:
! ./scripts/compile_cython.sh

running build_ext
cythoning features.pyx to features.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/casimir/casimir/data/named_entity_recognition/features.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
cythoning viterbi.pyx to viterbi.cpp
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/casimir/casimir/data/named_entity_recognition/viterbi.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'features' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/num

#Import The Data

In [0]:
#!gdown https://drive.google.com/uc?id=11FHyeH3kgBjv8UCT1iEvPzT8r_joJWXr
!gdown https://drive.google.com/uc?id=1arPlANNu8SY8ZgyDWKfxZiojkJonvdmk
import zipfile

with zipfile.ZipFile("Structured_Data.zip","r") as zip_ref:
    zip_ref.extractall("data/voc2007/")

In [0]:
with zipfile.ZipFile("Structured_Data.zip","r") as zip_ref:
    zip_ref.extractall("data/voc2007/")

In [0]:
https://drive.google.com/file/d/1arPlANNu8SY8ZgyDWKfxZiojkJonvdmk/view?usp=sharing

In [0]:

if not os.path.exists("outputs"):
    os.makedirs("outputs")

if not os.path.exists("outputs/loc"):
    os.makedirs("outputs/loc")

In [0]:
!python scripts/_run_one_loc.py dog

#Import Dependencies

In [0]:
from __future__ import absolute_import, division, print_function
from collections import defaultdict
import numpy as np
import os
import pickle as pkl
import sys
sys.path.append(os.path.abspath('./'))
import casimir.structured_prediction_experiment.utils as utils  # noqa: E402

In [0]:
# Script to save on feature loading time for localization experiments
l2reg = 10 #ℓ2 regularizer which taken to be l2 reg/n where n is the number of examples


obj_class = 'WBC' #Name of object class to run experiment on


parser = utils.make_parser(task='loc')
dim = 2304  # dimensionality of features
num_iter_smooth = 80 #Maximum number of passes to make through data
num_iter_nonsmooth = 80

# Change directories below
prefix = 'data/'
# 	Warm start strategy for Casimir-SVRG. 1 refers to prox-center while 2 to extrapolation and 3 to prev-iterate. Refer to documentation of casimir.optim.CasimirSVRGOptimizer for details.
warm_start_dict = {1: 'prox-center', 2: 'extrapolation', 3: 'prev-iterate'}

#Compile list of all tasks

In [0]:

for  warm_start in warm_start_dict.keys():
      tasks = []
      print(warm_start)
      common_smooth = ('--prefix {} --object-class {} --num_passes {} --K 10 ' +
                      '--warm_start  {}').format(prefix, obj_class, num_iter_smooth,warm_start)
      common_ns = '--prefix {} --object-class {} --num_passes {}'.format(prefix, obj_class, num_iter_nonsmooth)
      SEEDS = list(range(1, 10))

      # Choice of hyper-parameters
      # csvrg
      #Casimir-SVRG with learning rate specified implicitly via grad_lipshitz_parameter. Specify command line argument --L for grad_lipschitz_parameter. Requires smoothing.
      Ls_csvrg = { 'WBC': 4096.0}

      smoothers_csvrg = {'WBC': 1}

      # csvrg_lr: with adaptive smoothing
      #Casimir-SVRG with learning rate specified direcly as --lr. Requires smoothing.
      lrs_csvrg = {'WBC': 0.00048828125}

      smoothers_csvrg_lr = defaultdict(lambda: 10.0)

      # svrg
      #SVRG with learning rate specified via --lr. Requires smoothing.
      lrs_svrg = {'WBC': 0.00048828125}

      smoothers_svrg = {'WBC': 1.0}
      


      # sgd
      #GD with learning rates ηt=η0/(1+t/t0) where η0 is specified using --lr and t0 is specified using --lr-t and weighted averaging w¯T=2T(T+1)∑Tt=0t⋅wt
      lrs_sgd = {'WBC': 0.00048828125}

      t0_sgd = {'WBC': 1024.0}


      
          
      # Create arguments strings
      # CSVRG
      for seed in SEEDS:
          L = Ls_csvrg[obj_class]
          smoother = smoothers_csvrg[obj_class]
          task = ' --algorithm csvrg --l2reg {} --smoother {} --L {} --seed {}'.format(l2reg, smoother, L, seed)
          tasks += [common_smooth + task]

      

      # SVRG
      for seed in SEEDS:
          lr = lrs_svrg[obj_class]
          smoother = smoothers_svrg[obj_class]
          task = ' --algorithm svrg --l2reg {} --smoother {} --lr {} --seed {}'.format(l2reg, smoother, lr, seed)
          tasks += [common_smooth + task]

      # BCFW
      for seed in SEEDS:
          task = ' --algorithm bcfw --l2reg {} --seed {}'.format(l2reg, seed)
          tasks += [common_ns + task]

     

      # sgd
      for seed in SEEDS:
          lr = lrs_sgd[obj_class]
          t0 = t0_sgd[obj_class]
          task = (' --algorithm sgd --l2reg {}  --lr {} ' + 
                  '--lr-t {}  --seed {} ').format(l2reg, lr, t0, seed)
          tasks += [common_ns + task]
      outputs=[]
      # Run
      start = False
      train_ifo, dev_ifo, test_ifo = None, None, None
      for task in tasks:
          args = parser.parse_args(task.split())
          output_name = utils.get_output_filename(args, 'loc')

          if not start:
              start = True
              print('loading data')
              train_ifo, dev_ifo, test_ifo = utils.get_ifo(args, 'loc')
              
          dim = 2304
          initial_model = np.zeros(dim, dtype=np.float32)
          model, logs = utils.run_algorithm(args, initial_model, train_ifo, dev_ifo, test_ifo)
          outputs.append([logs, args])
          with open(output_name, 'wb') as fn:
              pkl.dump([logs, args], fn, protocol=2)
          print('Done Training')
          print(warm_start_dict[warm_start])
      np.save(warm_start_dict[warm_start],outputs)


3
loading data
Loaded dataset: labels 0.70s ; images 2.78s.
Loaded dataset: labels 0.64s ; images 1.78s.
l2reg = 0.038910505836575876
Starting optimization with csvrg
Epoch		Function		Dev_function	Dev_evaluation		Time
0.00		0.83432388		0.832093		0.024743	0.000000	0.014407		0.39
1.00		0.77010128		0.885359		0.396686	0.383721	0.323235		1.12
2.00		0.70248872		0.919844		0.380087	0.395349	0.309787		1.15
3.00		0.65610374		0.929358		0.371449	0.360465	0.317867		1.12
4.00		0.62318783		0.937053		0.376234	0.383721	0.316543		1.15
5.00		0.59682067		0.953243		0.375546	0.383721	0.309900		1.17
6.00		0.57485170		0.963033		0.369349	0.348837	0.307526		1.18
7.00		0.55624735		0.979373		0.364083	0.337209	0.303437		1.13
8.00		0.53927807		0.989998		0.366120	0.360465	0.297089		1.12
9.00		0.52510144		1.001656		0.367490	0.348837	0.295448		1.20
10.00		0.51086094		1.014900		0.382093	0.372093	0.291822		1.16
11.00		0.49829193		1.027575		0.375972	0.360465	0.287729		1.14
12.00		0.48682030		1.037431		0.378476	0.372093	0

# Visualizing Of The Results

In [0]:
np_load_old = np.load

    # modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

from matplotlib import pyplot as plt


if not os.path.exists("plot"):
    os.makedirs("plot")


for key in warm_start_dict.keys():
  outputs=df=np.load(warm_start_dict[key]+".npy") [None][0]
  names=[]
  for i in range(len(outputs)):
   if not  outputs[i,1].algorithm in names:
    names.append(outputs[i,1].algorithm)
  
  experiments={}
  for i in range(len(names)):
    tmp=[]
    for j in range(len(outputs)):
      if outputs[j,1].algorithm == names[i]:
        tmp.append(outputs[j,0])
    experiments[names[i]]=np.array(tmp)
  tags=["Train Loss","Val CorLoc "]
  

  colors=['C'+str(i) for i in  range(10)]
  for j,k in enumerate([1,3]):
    for i in range(len(names)):
        errors=experiments[names[i]][:,:,k]
        mean =errors.mean(axis=0)
        std = errors.std(axis=0)
        
        plt.plot(np.arange(len(mean)), mean, colors[i], linewidth = 2,label=names[i])
        plt.plot(np.arange(len(mean)), mean-std, colors[i]+'--')
        plt.plot(np.arange(len(mean)), mean+std, colors[i]+'--')
        plt.legend()
    
    plt.ylabel(tags[j])
    plt.xlabel("Number of iteration")
    plt.savefig(warm_start_dict[key]+"_"+tags[j]+".png")
    plt.close()


np.load = np_load_old